In [ ]:
import pandas as pd
#import pylab
#import scipy
import matplotlib.pyplot as plt
import matplotlib #so I can call next line
matplotlib.style.use('ggplot')
import seaborn as sns
import numpy as np

#visuals
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format #7,123,001.34
#'{:20,.2f}'.format #change pandas display format
pd.options.display.max_rows = 20 
#pd.get_option("display.max_rows")


from __future__ import division #so I can have float as std and int as //

#my own functions
#from supportFunctions import * #you CAN'T RELOAD functions

# Processing GNSS measurements

This presentation aims to explain how to calculate pseudo ranges from the obtainable from the Android API 24+.

This code is the equivalent of `ProcessGnssMeas.m` Matlab code from [Google GPS Measurement Tools](https://github.com/google/gps-measurement-tools).

## short intro to GNSS ranges

<img src="https://upload.wikimedia.org/wikipedia/commons/c/c3/Bad_gdop.png", width=50%,height=60>


Pseudorange is transmission time from satellite to the receiver, calculated as $$L = \frac{T_{receiver \atop arrival}
-T^{satellite \atop transmission}}{c}$$. 
It is called pseudorange as <https://www.youtube.com/playlist?list=PL635065C128E0DB27>




# Loading and reading data

Android data read by GNSSLogger output:
* Position, Velocity and Time (PVT) solution 
* ephemeris information *(not yet implemented)*
* raw rages information

For this workshop we will only focus on the last one.

In [ ]:
#in-memory grep implementation, filter lines starting wth filterKeyword
#http://stackoverflow.com/questions/10717504/is-it-possible-to-use-read-csv-to-read-only-specific-lines
def SimpleLineGrep(ASCIIfileName,filterKeyword):
  from cStringIO import StringIO
  s = StringIO()

  with open(ASCIIfileName) as f:
      for line in f:
          if line.startswith(filterKeyword):
              s.write(line)
  s.seek(0) # "rewind" to the beginning of the StringIO object
  outFile = open("%s/dmp.csv" % ASCIIfileName[:ASCIIfileName.index('/')], "w")
  outFile.write(s.getvalue())
  outFile.close()

  return s

In [ ]:
data_file = "./sampleData/workshop_trials01.txt"
print("filtering PR from %s" % data_file)
RawMeas = SimpleLineGrep(data_file,'Raw')

colNames = ["Raw","ElapsedRealtimeMillis","TimeNanos","LeapSecond","TimeUncertaintyNanos","FullBiasNanos","BiasNanos","BiasUncertaintyNanos","DriftNanosPerSecond","DriftUncertaintyNanosPerSecond","HardwareClockDiscontinuityCount","SVid","TimeOffsetNanos","State","ReceivedSvTimeNanos","ReceivedSvTimeUncertaintyNanos","Cn0DbHz","PseudorangeRateMetersPerSecond","PseudorangeRateUncertaintyMetersPerSecond","AccumulatedDeltaRangeState","AccumulatedDeltaRangeMeters","AccumulatedDeltaRangeUncertaintyMeters","CarrierFrequencyHz","CarrierCycles","CarrierPhase","CarrierPhaseUncertainty","MultipathIndicator","SnrInDb","ConstellationType"]
df_GNSS = pd.read_csv("dmp.csv", delimiter = ",",error_bad_lines=False,header=None,usecols=range(1,len(colNames)),
                     names= colNames,encoding = 'utf-8-sig',na_values = ["NULL",""],engine ='c')
print df_GNSS.dtypes
df_GNSS.head()

In [ ]:
GNSS_const = {'totalWeekSecs':7*24*3600,'lightSpeed':299792458}

In [ ]:
def BasicInfo(AndroidData):

  listOfSV = df_GNSS.SVid.unique()
  listOfConstelations = df_GNSS.ConstellationType.unique()
  GNSS_Constelations = {1:'GPS',2:'SBAS',3:'GLONASS',4:'QZSS',5:'BeiDou',6:'Galileo'}

  print 'Observing following SVs:{}\nObserved constelations: {}'.format(
      ','.join(map(str,listOfSV)),','.join([GNSS_Constelations[s] for s in listOfConstelations]))

## First look
Let's explore [Android GNSS status](https://developer.android.com/reference/android/location/GnssStatus.html) API.


In [ ]:
BasicInfo(df_GNSS)

For simplicity we will focus on a single constelation.

In [ ]:
df_GNSS = df_GNSS[df_GNSS.ConstellationType==1]
BasicInfo(df_GNSS)

## Getting pseudoranges

* verify that we obtained TOW before calculating PR
* receiver clock [ns] is calculated from `public long getTimeNanos()`
* received GNSS satelite time [ns]  is calculated from `public long getReceivedSvTimeNanos()`
* anything within 1ms is considered same epoch



[Frank sample](https://docs.google.com/spreadsheets/d/1e9JaVn9hq04tEVoCDkO7EHi8LdDyf0X7_IuQZ3U1b-E/edit#gid=80118229)

In [ ]:
state = df_GNSS.State.iloc[0]
print 'Or in binary: {0:#010b}'.format(state)
print '1st bit{:#010b}\n3rd bit{:#010b}'.format(1<<0,1<<2)
#state = 0b00101110
print 'TOW flag set: {}'.format((state & (1 << 0))!=0 and (state & (1 << 2))!=0)

In [ ]:
len(df_GNSS)

In [ ]:
df_GNSS.FullBiasNanos

In [ ]:
GPSWeek = (-df_GNSS.FullBiasNanos*1e-9/GNSS_const['totalWeekSecs']).astype('int')
print 'GPS week {} '.format(GPSWeek.unique())

tRx_ns  = df_GNSS.TimeNanos-df_GNSS.FullBiasNanos.iloc[0]-(GPSWeek*GNSS_const['totalWeekSecs']*1e9)  +df_GNSS.TimeOffsetNanos 
#tRx_ns  = df_GNSS.TimeNanos-df_GNSS.FullBiasNanos-(GPSWeek*GNSS_const['totalWeekSecs']*1e9)  +df_GNSS.TimeOffsetNanos 
PR_m = (tRx_ns-df_GNSS.ReceivedSvTimeNanos)*299792458*1e-9
PR_m.tail()

In [ ]:
# sp1
plt.subplot(121)
plt.scatter(tRx_ns,df_GNSS.ReceivedSvTimeNanos,);
plt.axis('equal')

# sp2
plt.subplot(122)
plt.boxplot([tRx_ns,df_GNSS.ReceivedSvTimeNanos]);

#diff = df_GNSS.ReceivedSvTimeNanos-tRx_ns
#plt.plot(diff[0:100])

In [ ]:
df_GNSS.columns

## Visualising ranges

* compute full cycle time of measurement, in milliseonds (see `ReadGnssLogger.m`)
* split ranges by SV_ID
* plot


In [ ]:
listOfSV = df_GNSS.SVid.unique()
listOfSV

In [ ]:
allRxSec = (df_GNSS.TimeNanos - df_GNSS.FullBiasNanos)*1e-9;
epoch=allRxSec[0::len(listOfSV)]

In [ ]:
df_PR =pd.DataFrame({'epoch': allRxSec,'SV_ID': df_GNSS.SVid,'PR': PR_m})
SV_ranges = df_PR.pivot(index='epoch',columns='SV_ID', values='PR')
SV_ranges.plot();

## Problems with the accuracy of calculations


In [ ]:
def CheckCalculus(number):

  y=number-1
  z=number-1+1
  print 'x-y={:}\nz-x={:}'.format(x-y,x-y)

In [ ]:
x=-1151285108458178048
CheckCalculus(x)

In [ ]:
CheckCalculus(x*1e10)

Some solutions:

* <http://mpmath.org/>